# Anime Recommendation System
### Predicting which animes you'll like
---

## 1.0 Imports

In [66]:
#Data extraction and manipulation
import pandas as pd 
import pymysql as pml
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import numpy as np

#Machine Learning Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#Evaluation Metrics
from sklearn.metrics import roc_auc_score, average_precision_score

#Data Visualization
from sklearn.tree import plot_tree
import matplotlib.pylab as plt
import seaborn as sns

# 2.0 Helper Functions

In [67]:
# disable max columns limit
pd.set_option('display.max_columns', 70)

# sets to not crop long vlaues for rows
pd.set_option("display.max_colwidth", 70)

# sets seaborn default style
sns.set_style('white')

# 3.0 Check the Dataset

## 3.1 Data Dictionary

The dataset that i am using is from the site myanimelist: https://myanimelist.net/topanime.php

extracted via API, filtered and stored in a database on these notebooks: 


 |  **Feature** 	|                  **Description**                 	|
 |:------------:	|:------------------------------------------------:	|
 |   anime_id   	|          An id that represents a animes          	|
 |     title    	|                  The anime title                 	|
 |     genre    	|              The main genre of anime             	|
 |  mean_score  	|          Average anime viewership score          	|
 |  media_type  	| Anime type (movie, tv, special, oca, ona, music) 	|
 | num_episodes 	|              The number of episodes              	|
 |    picture   	|       A link to view the anime front cover       	|
 |    rating    	|        Anime rating (pg_13, r, g, r+, pg)        	|
 |  start_date  	|                   Release date                   	|
 |   end_date   	|     Completion date (may be a expected date)     	|
 |       y      	|      My interest in anime (0 - no, 1 - yes)      	| 


## 3.2 Loading the dataset

In [68]:
#Read the data with labels and load into a dataframe 
df_animes_with_labels = pd.read_csv('animes_with_label.csv', index_col=0)
#keep only those with label
df_animes_with_labels = df_animes_with_labels[df_animes_with_labels['y'].notnull()] 
df_animes_with_labels.shape

(605, 11)

In [69]:
#Creating a new dataframe with the new labeled data 
df_active_learning = pd.read_csv("active_label_done.csv", index_col= 0)
df_active_learning = df_active_learning[df_active_learning['y'].notnull()]
df_active_learning.shape

(200, 12)

In [70]:
def data_split(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                        random_state=42)
    return X_train, X_test, y_train, y_test

def X_split(X, test_size):
    X_train, X_test = train_test_split(X, test_size=test_size,
                                                        random_state=42)
    return X_train, X_test

In [71]:
def word_vectorizer(word_train, word_test, min_df):
    word_vec = TfidfVectorizer(min_df= min_df)
    word_bow_train = word_vec.fit_transform(word_train)
    word_bow_test = word_vec.transform(word_test)
    
    return word_bow_train, word_bow_test

In [72]:
#Split the data with labels
X = df_animes_with_labels[['mean_score', 'num_episodes']]
y = df_animes_with_labels['y'].copy()

X_train, X_test, y_train, y_test = data_split(X, y, 0.5)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((302, 2), (303, 2), (302,), (303,))

In [73]:
#Split the titles in training and test
title_train, title_test = X_split(df_animes_with_labels['title'], 0.5)

In [74]:
title_bow_train, title_bow_test = word_vectorizer(title_train, title_test, 1)

In [75]:
Xtrain = hstack([X_train, title_bow_train])
Xtest = hstack([X_test, title_bow_test])

In [76]:
#Concat the new data to test set
X_test_plus = pd.concat([X_test, df_active_learning[['mean_score', 'num_episodes']]])
y_test_plus = pd.concat([y_test, df_active_learning['y']])
title_test_plus = pd.concat([title_test, df_active_learning['title']])

In [77]:
title_bow_train, title_bow_test_plus = word_vectorizer(title_train, title_test_plus, 1)

In [78]:
Xtest_plus = hstack([X_test_plus, title_bow_test_plus])

In [79]:
mdl = RandomForestClassifier(n_estimators = 1000, random_state= 0, class_weight="balanced")
mdl.fit(Xtrain, y_train)
probability_score_test_plus = mdl.predict_proba(Xtest_plus)[:,1]

In [80]:
print(average_precision_score(y_test_plus, probability_score_test_plus), 
      roc_auc_score(y_test_plus, probability_score_test_plus))

0.6149893004819301 0.7838457336818696


In [81]:
#Testing min_samples_leaf = 2
mdl = RandomForestClassifier(n_estimators = 1000, random_state= 0, min_samples_leaf=2, class_weight="balanced")
mdl.fit(Xtrain, y_train)
probability_score_test_plus = mdl.predict_proba(Xtest_plus)[:,1]

print(average_precision_score(y_test_plus, probability_score_test_plus), 
      roc_auc_score(y_test_plus, probability_score_test_plus))

0.4808257084083387 0.6906577971987048


In [82]:
#Testing min_samples_leaf = 2 and n_estimator = 500
mdl = RandomForestClassifier(n_estimators = 500, random_state= 0, min_samples_leaf=2, class_weight="balanced")
mdl.fit(Xtrain, y_train)
probability_score_test_plus = mdl.predict_proba(Xtest_plus)[:,1]

print(average_precision_score(y_test_plus, probability_score_test_plus), 
      roc_auc_score(y_test_plus, probability_score_test_plus))

0.47144530034091076 0.6842300339432719


In [83]:
#Testing tfdfvectorizer ngram_range = (1,3)

def word_vectorizer(word_train, word_test, min_df):
    word_vec = TfidfVectorizer(min_df= min_df, ngram_range=(1,3))
    word_bow_train = word_vec.fit_transform(word_train)
    word_bow_test = word_vec.transform(word_test)
    
    return word_bow_train, word_bow_test

#Split the titles in training and test
title_train, title_test = X_split(df_animes_with_labels['title'], 0.5)

title_bow_train, title_bow_test = word_vectorizer(title_train, title_test, 1)

Xtrain = hstack([X_train, title_bow_train])
Xtest = hstack([X_test, title_bow_test])

#Concat the new data to test set
X_test_plus = pd.concat([X_test, df_active_learning[['mean_score', 'num_episodes']]])
y_test_plus = pd.concat([y_test, df_active_learning['y']])
title_test_plus = pd.concat([title_test, df_active_learning['title']])

title_bow_train, title_bow_test_plus = word_vectorizer(title_train, title_test_plus, 1)

Xtest_plus = hstack([X_test_plus, title_bow_test_plus])

mdl = RandomForestClassifier(n_estimators = 1000, random_state= 0, class_weight="balanced")
mdl.fit(Xtrain, y_train)
probability_score_test_plus = mdl.predict_proba(Xtest_plus)[:,1]

print(average_precision_score(y_test_plus, probability_score_test_plus), 
      roc_auc_score(y_test_plus, probability_score_test_plus))

0.6876159471901129 0.8117611486091061


## LightGBM

In [84]:
from lightgbm import LGBMClassifier

In [85]:
mdl = LGBMClassifier(random_state=0, class_weight= 'balanced', n_jobs=6)
mdl.fit(Xtrain, y_train)
probability_score_test_plus = mdl.predict_proba(Xtest_plus)[:,1]

C:\Users\Luís Eduardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [86]:
print(average_precision_score(y_test_plus, probability_score_test_plus), 
      roc_auc_score(y_test_plus, probability_score_test_plus))

0.4263935604056547 0.6575533533611642


## Bayesian Optimization

In [87]:
from skopt import forest_minimize

In [146]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    
    title_train, title_test = X_split(df_animes_with_labels['title'], 0.5)
    
    word_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    word_bow_train = word_vec.fit_transform(title_train)
    word_bow_test = word_vec.transform(title_test)

    Xtrain = hstack([X_train, word_bow_train])
    Xtest = hstack([X_test, word_bow_test])

    #Concat the new data to test set
    X_test_plus = pd.concat([X_test, df_active_learning[['mean_score', 'num_episodes']]])
    y_test_plus = pd.concat([y_test, df_active_learning['y']])
    title_test_plus = pd.concat([title_test, df_active_learning['title']])

    word_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    word_bow_train = word_vec.fit_transform(title_train)
    word_bow_test_plus = word_vec.transform(title_test_plus)
    
    Xtest_plus = hstack([X_test_plus, word_bow_test_plus])
    
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    
    mdl.fit(Xtrain, y_train)
    
    probability_score_test_plus = mdl.predict_proba(Xtest_plus)[:,1]
    
    print(roc_auc_score(y_test_plus, probability_score_test_plus))
    
    return -average_precision_score(y_test_plus, probability_score_test_plus)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (1,5), # min_df
          (1,5)] # ngram_range

res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\Luís Eduardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
C:\Users\Luís Eduardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.7203484062268347
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.3101
Function value obtained: -0.5002
Current minimum: -0.5002
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.6912137645819516
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0908
Function value obtained: -0.4697
Current minimum: -0.5002
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\Luís Eduardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.672164566345441
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.1027
Function value obtained: -0.4806
Current minimum: -0.5002
Iteration No: 4 started. Evaluating function at random point.
[0.0014099928811969545, 9, 9, 0.6502182010234373, 0.6866210554187129, 828, 5, 2]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


...

0.7452596465217901
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 0.7511
Function value obtained: -0.5878
Current minimum: -0.6315
Iteration No: 49 started. Searching for the next optimal point.
[0.0010499941434957431, 6, 1, 0.33718308544705317, 0.17381253387200854, 559, 3, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\Luís Eduardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.7467714876516717
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 0.7252
Function value obtained: -0.5895
Current minimum: -0.6315
Iteration No: 50 started. Searching for the next optimal point.
[0.0014981253289572797, 7, 1, 0.28904027807001825, 0.06063151338681717, 824, 5, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\Luís Eduardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.8035972065077446
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.8923
Function value obtained: -0.7105
Current minimum: -0.7105


In [148]:
res.x

[0.0014981253289572797,
 7,
 1,
 0.28904027807001825,
 0.06063151338681717,
 824,
 5,
 1]

In [147]:
res.fun

-0.7105097462366898

## Logistic Regression

In [91]:
from sklearn.preprocessing import MaxAbsScaler
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression

In [167]:
Xtrain2 = csr_matrix(Xtrain.copy())
Xtest2 = csr_matrix(Xtest_plus.copy())

scaler = MaxAbsScaler()

Xtrain2 = scaler.fit_transform(Xtrain2)
Xtest2 = scaler.transform(Xtest2)

In [174]:
mdl = LogisticRegression(C=10,n_jobs=6, random_state=0)
mdl.fit(Xtrain2, y_train)

LogisticRegression(C=10, n_jobs=6, random_state=0)

In [175]:
p = mdl.predict_proba(Xtest2)[:, 1]

In [170]:
# C = 500
print(average_precision_score(y_test_plus, p), 
      roc_auc_score(y_test_plus, p))

0.644108514437606 0.7950723732979595


In [173]:
#C = 200
print(average_precision_score(y_test_plus, p), 
      roc_auc_score(y_test_plus, p))

0.6439463286580266 0.7957941555148064


In [176]:
# C = 10
print(average_precision_score(y_test_plus, p), 
      roc_auc_score(y_test_plus, p))

0.6352147464960527 0.7898833443876556


# Final Emsemble

## Random Forest

In [155]:
mdl_random = RandomForestClassifier(n_estimators = 1000, random_state= 0, class_weight="balanced")
mdl_random.fit(Xtrain, y_train)
probability_score_random_forest = mdl_random.predict_proba(Xtest_plus)[:,1]

print(average_precision_score(y_test_plus, probability_score_random_forest), 
      roc_auc_score(y_test_plus, probability_score_random_forest))

0.6876159471901129 0.8117611486091061


## LightGBM

In [157]:
params = [0.0014981253289572797, 7, 1, 0.28904027807001825, 0.06063151338681717, 824, 1, 3]

lr = params[0]
max_depth = params[1]
min_child_samples = params[2]
subsample = params[3]
colsample_bytree = params[4]
n_estimators = params[5]
    
min_df = params[6]
ngram_range = (1, params[7])
    
    
title_train, title_test = X_split(df_animes_with_labels['title'], 0.5)
    
word_vec = TfidfVectorizer(min_df=min_df, ngram_range= ngram_range)
word_bow_train = word_vec.fit_transform(title_train)
word_bow_test = word_vec.transform(title_test)

Xtrain = hstack([X_train, word_bow_train])
Xtest = hstack([X_test, word_bow_test])

#Concat the new data to test set
X_test_plus = pd.concat([X_test, df_active_learning[['mean_score', 'num_episodes']]])
y_test_plus = pd.concat([y_test, df_active_learning['y']])
title_test_plus = pd.concat([title_test, df_active_learning['title']])

word_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
word_bow_train = word_vec.fit_transform(title_train)
word_bow_test_plus = word_vec.transform(title_test_plus)
    
Xtest_plus = hstack([X_test_plus, word_bow_test_plus])
    
mdl_lgbm = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    
mdl_lgbm.fit(Xtrain, y_train)
    
probability_score_light = mdl_lgbm.predict_proba(Xtest_plus)[:,1]
    
print(average_precision_score(y_test_plus, probability_score_light), roc_auc_score(y_test_plus, probability_score_light))

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.7105097462366898 0.8035972065077446


C:\Users\Luís Eduardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


## Logistic Regression

In [177]:
from sklearn.pipeline import make_pipeline

In [178]:
Xtrain2 = csr_matrix(Xtrain.copy())
Xtest2 = csr_matrix(Xtest_plus.copy())

lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(C = 500, n_jobs= 6, random_state= 0))

lr_pipeline.fit(Xtrain2, y_train)

probability_score_logistic = lr_pipeline.predict_proba(Xtest2)[:, 1] 

print(average_precision_score(y_test_plus, probability_score_logistic), 
      roc_auc_score(y_test_plus, probability_score_logistic))

0.644108514437606 0.7950723732979595


## Ensemble

0.6876159471901129 0.8117611486091061 RF

0.7105097462366898 0.8035972065077446 LGBM

0.644108514437606 0.7950723732979595 LR

In [102]:
probability_score = (probability_score_light + probability_score_random_forest + probability_score_logistic) / 3
print(average_precision_score(y_test_plus, probability_score), roc_auc_score(y_test_plus, probability_score))

0.7132421902362809 0.8227927119503726


In [103]:
pd.DataFrame({"LR" : probability_score_logistic, "RF": probability_score_random_forest, "LGBM": probability_score_light}).corr()

,LR,RF,LGBM
LR,1.000000,0.717407,0.556873
RF,0.717407,1.000000,0.805850
LGBM,0.556873,0.805850,1.000000


In [104]:
probability_score = 0.5 * probability_score_logistic + 0.5 * probability_score_light
print(average_precision_score(y_test_plus, probability_score), roc_auc_score(y_test_plus, probability_score))

0.7088473966317359 0.8183059576294331


In [105]:
probability_score = 0.4 * probability_score_logistic + 0.6 * probability_score_light
print(average_precision_score(y_test_plus, probability_score), roc_auc_score(y_test_plus, probability_score))

0.7174079743934428 0.8215442237915025


In [106]:
probability_score = 0.3 * probability_score_logistic + 0.7 * probability_score_light
print(average_precision_score(y_test_plus, probability_score), roc_auc_score(y_test_plus, probability_score))

0.7231399030576191 0.8236705551870782


In [107]:
probability_score = 0.2 * probability_score_logistic + 0.8 * probability_score_light
print(average_precision_score(y_test_plus, probability_score), roc_auc_score(y_test_plus, probability_score))

0.7306053772507798 0.8252701806406305


In [108]:
probability_score = 0.1 * probability_score_logistic + 0.9 * probability_score_light
print(average_precision_score(y_test_plus, probability_score), roc_auc_score(y_test_plus, probability_score))

0.7405124135918438 0.8269283289766298


In [109]:
probability_score = 0.2 * probability_score_random_forest + 0.8 * probability_score_light
print(average_precision_score(y_test_plus, probability_score), roc_auc_score(y_test_plus, probability_score))

0.7248197765306668 0.8210760407319262


## Save Models

In [179]:
import joblib as jb

jb.dump(mdl_lgbm, "lgbm_animes.pkl.z")
jb.dump(lr_pipeline, "Logistic_reg_animes.pkl.z")
jb.dump(word_vec, "word_vectorizer_animes.pkl.z")

['word_vectorizer_animes.pkl.z']